In [2]:
%matplotlib inline
from brian2 import *
import matplotlib.pyplot as plt
import numpy as np
import scipy.signal as signal

In [4]:
# clear_cache('cython')

In [6]:
defaultclock.dt = 0.01*ms

In [7]:
#### Borrowing from input factory of neurodynex package

def get_step_current(t_start, t_end, t_total, unit_time, amplitude):

    I_arr = np.zeros(t_total)*amp
    I_arr[t_start : t_end+1] = amplitude
    t_arr = TimedArray(np.arange(0, t_total), dt = 1*unit_time)
    I_arr = TimedArray(I_arr, dt = 1*unit_time)
    return t_arr, I_arr

In [5]:
ps1 = {
    "CABLE_LENGTH" : 500. * um,  # length of dendrite
    "CABLE_DIAMETER" : 2. * um,  # diameter of dendrite
    "R_LONGITUDINAL" : 0.5 * kohm * mm,  # Intracellular medium resistance
    "R_TRANSVERSAL" : 1.25 * Mohm * mm ** 2,  # cell membrane resistance (->leak current)
    "E_LEAK" : -70. * mV,  # reversal potential of the leak current (-> resting potential)
    "CAPACITANCE" : 0.8 * uF / cm ** 2
}

In [8]:
def passive_cable(I_arr, I_loc, params, duration, initial_voltage, nr_compartments):
    length=params["CABLE_LENGTH"]
    diameter=params["CABLE_DIAMETER"]
    r_longitudinal=params["R_LONGITUDINAL"]
    r_transversal=params["R_TRANSVERSAL"]
    e_leak=params["E_LEAK"]
    capacitance=params["CAPACITANCE"]

    cable_morphology = Cylinder(diameter=diameter, length=length, n=nr_compartments)
    
    EL = e_leak
    RT = r_transversal
    eqs = """
    Iext = current(t, location_index): amp (point current)
    location_index : integer (constant)
    Im = (EL-v)/RT : amp/meter**2
    """
    cable_model = SpatialNeuron(morphology=cable_morphology, model=eqs, Cm=capacitance, Ri=r_longitudinal)
    monitor_v = StateMonitor(cable_model, "v", record=True)

    # inject all input currents at the specified location:
    nr_input_locations = len(I_loc)
    input_current_0 = np.insert(I_arr.values, 0, 0., axis=1) * amp  # insert default current: 0. [amp]
    current = TimedArray(input_current_0, dt=I_arr.dt * second)
    for current_index in range(nr_input_locations):
        insert_location = I_loc[current_index]
        compartment_index = int(np.floor(insert_location / (length / nr_compartments)))
        # next line: current_index+1 because 0 is the default current 0Amp
        cable_model.location_index[compartment_index] = current_index + 1

    # set initial values and run for 1 ms
    cable_model.v = initial_voltage
    run(duration)
    return monitor_v, cable_model

    